# Herval Deep Mailing

In [ ]:
import pandas as pd
import dateutil.parser as parser
import os.path
import math
import logging
import numpy as np
from multiprocessing import Pool
from datetime import datetime
import gc
import pickle

In [ ]:
log_location = "../../logs/"
arquivo_fonte = "../../data/batch03/inputs/herval_final.xls"
arquivo_saida = "../../data/batch03/intermediate/Herval.normalized.pickle"


pd.options.display.max_columns = 150
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
logger = logging.getLogger()
logger.handlers = []
logger = logging.getLogger(__name__)
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'prepare_data.log.' + \
                                          datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

def print_log(x):
    logging.debug(x)
    print(x)

In [ ]:
def limpar_df(pagantes):
    del pagantes['CPF']

    return pagantes

In [ ]:
def func_str(x):
    return str(x).replace(" ","_").upper().strip()

def func_strip(x):
    return str(x).strip()

def func_start_ALTA(x):
    return str(x).startswith('ALTA')

In [ ]:
def ConverterPAGOU(x):
    return 1 if x == 1 else 0
def ConverterSimNao(x):
    return 1 if x == "Sim" else 0

In [ ]:
converters = { 
    "ESCOLARIDADE" : func_str
}

df_dtypes= {

}

In [ ]:
def print_colunas(df):
    for coluna in pagantes.head(10).columns.values:
        print_log(coluna)    

In [ ]:
pagantes = pd.read_excel(arquivo_fonte, dtype=df_dtypes, converters = converters)
print_log("CSV carregado, limpando colunas desnecessarias")
print_colunas(pagantes)
pagantes = limpar_df(pagantes)
print_log("-------------------------------------------")
print_log("Colunas Remanescentes...")
print_log("-------------------------------------------")
print_colunas(pagantes)

In [ ]:
print_log(pagantes.head(10))

In [ ]:
def ParseNumTelefone(x):
    ret = np.nan
    if str(x).startswith('['):
        ret = x[1:]
        ret = ret.split(';')[0]
        ret = ret[0:2]
    return ret

def ParseRendaEstimada(x):
    ret = np.nan
    if str(x) == 'SEM INFORMACAO':
        return ret
    if len(str(x)) > 4:
        ret = str(x)
        ret = ret.replace('DE ',' ').replace(' A ', ' ').replace('SEM INFORMACAO', ' ').replace('ACIMA ', ' ').replace(',01',' ').replace('.','').replace('ATE ',' ')
        ret = ret.replace('  ',' ')
        ret = ret.strip()
        ret = ret.split(' ')[0]
        ret = int(ret)
    return ret

In [ ]:
def CreateColumnStr(cols, df, source_col):
    for col in cols:
        if str(col) == "nan":
            continue
        df['NORM_' + source_col + "_" + str(col)] = df.apply(lambda row: 1 if func_str(row[source_col]) == func_str(col) else 0, axis=1)
    return df

def RemoveOutliers(df, source_col, min, max):
    df['NORM_' + source_col] = df.apply(lambda row: 0   if row[source_col] < min else row[source_col] , axis=1)
    df['NORM_' + source_col] = df.apply(lambda row: max if row[source_col] > max else row[source_col] , axis=1)
    return df

def CreateProportion(df, source_col):
    max = df[source_col].mean() + (df[source_col].std() * 4 )
    min = 0
    
    df = RemoveOutliers(df, source_col, min, max)
    print_log("({}) - {}, {}, {}".format(source_col, max,min, df[source_col].mean()))
   
    df['NORM_' + source_col] = df.apply(lambda row: 0 if row['NORM_' + source_col] == 0 else ((row['NORM_' + source_col] - min) / max) , axis=1)
    return df

def CreateNumeroTelefone(df,source_col):
    df['PARSED_' + source_col] = df.apply(lambda row: ParseNumTelefone(row[source_col]) , axis=1)
    return df

def CreateRendaEstimada(df,source_col):
    df['PARSED_' + source_col] = df.apply(lambda row: ParseRendaEstimada(row[source_col]) , axis=1)
    return df

In [ ]:
pagantes = pagantes.query("CONTRATO_ATRASO < 600 and CONTRATO_ATRASO > 0 and CLIENTE_VALOR_DIVIDA < 1000")
print(pagantes.CONTRATO_ATRASO.max())

In [ ]:
pagantes['PAGOU'] = pagantes.apply(lambda row: 1 if row['PAGOU'] == 1 else 0, axis=1)

In [ ]:
pagantes = CreateProportion(pagantes, 'CLIENTE_VALOR_DIVIDA')
pagantes = CreateProportion(pagantes, 'CONTRATO_ATRASO')
#pagantes = CreateRendaEstimada(pagantes,'RENDA_ESTIMADA')
pagantes = CreateProportion(pagantes, 'RENDA_PRESUMIDA')
pagantes = CreateColumnStr(pagantes['ESCOLARIDADE'].unique(), pagantes, 'ESCOLARIDADE')
pagantes = CreateColumnStr(pagantes['CLASSE_SOCIAL'].unique(), pagantes, 'CLASSE_SOCIAL')

#pagantes.head(200)

In [ ]:
colunas_para_processar = ['PAGOU']
for coluna in pagantes.head(10).columns.values:
    if not coluna.startswith('NORM_'):
        continue
    colunas_para_processar.append(coluna)
colunas_para_processar = sorted(list(set(colunas_para_processar)))

pagantes = pagantes[colunas_para_processar]
pagantes = pagantes.dropna(axis = 0, how="any")
print_log("Nao pagantes:{}".format(len(pagantes[pagantes.PAGOU == 0])))
print_log("Pagantes:{}".format(len(pagantes[pagantes.PAGOU == 1])))

pagantes.head(10)

In [ ]:
pagantes.to_pickle(arquivo_saida)